In [1]:
import pandas as pd

filename = 'PI_DataSet_6_2019.txt'
f = open(filename)
for line in f:
    header = line.split()
    start_ind = header.index('P1')
    end_ind = header.index('P99')+1
    header = header[:end_ind]
    df = pd.DataFrame(columns = header)
    break  
cnt = 0
for line in f:
    l = line.split('\t')[:end_ind]
   
    df.loc[cnt]=l
    cnt += 1
f.close()


In [2]:
print("SeqID is unique")
assert len(df['SeqID']) == len(set(df['SeqID']))

SeqID is unique


In [3]:
def isfloat_or_na(num):
    try:
        a = float(num)
        return True
    except:
        if num == 'NA':
            return True
        return False
inhib_start = 1

In [4]:
print("The inhibitors are either NA or a float")
for inh in header[inhib_start:start_ind]:
    assert filter(lambda x: not isfloat_or_na(x),df[inh] )==[]

The inhibitors are either NA or a float


In [5]:
import re
# amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
def is_only_aa(strg, search = re.compile(r'[^ACDEFGHIKLMNPQRSTVWY]').search):
    return not bool(search(strg))
'''
test_subjects = ['A','B','A*',"-","ACDEF"]
for t in test_subjects:
    print(t,is_only_aa(t))
'''

'\ntest_subjects = [\'A\',\'B\',\'A*\',"-","ACDEF"]\nfor t in test_subjects:\n    print(t,is_only_aa(t))\n'

In [8]:
letters = list("PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKMIGGIGGFIKVRQYDQILIEICGHKAIGTVLVGPTPVNIIGRNLLTQIGCTLNF")

In [9]:
print ("Take * out, substitute -, . and X with the letters")

Take * out, substitute -, . and X with the letters


In [10]:
for i,name in enumerate(header[start_ind:]):
    #print filter(lambda x:not is_only_aa(x), set(df[name]))
    df[name] = df[name].replace("X","-").replace(".","-").str.replace("*","").replace("-",letters[i])
    # print filter(lambda x:not is_only_aa(x), set(df[name]))

In [11]:
odds = set()
for pi in header[start_ind:]:
    odds.update(filter(lambda x:not is_only_aa(x), set(df[pi])))
    #print filter(lambda x:not is_only_aa(x), set(df[pi]))
assert odds == set()

In [12]:
def combs(los, ci):
    if ci == len(los)-1:
        return list(los[-1])
    else:
        prev_results = combs(los,ci+1)
        return [l+r for r in prev_results for l in los[ci]]

In [13]:
expanded_data = []
prev_count = 0

for ind,d in enumerate(df.values.tolist()):
    inds = [i for i in range(start_ind,end_ind) if len(d[i])>1]
    if inds:
        los = [d[i] for i in inds]
        changes = combs(los,0)
        #print los, changes
        for c in changes:
            c = list(c)
            for j,k in enumerate(inds):
                if c[j]=="":
                    print d
                    break
                d[k] = c[j]
            expanded_data.append(tuple(d))
            # token =  (",".join(d))+"\n"
            # f.write(token)
            prev_count += 1
    else:
        #token =  (",".join(d))+"\n"
        #f.write(token)
        expanded_data.append(tuple(d))
        prev_count += 1

In [14]:
P = [[] for _ in range(99)]
for d in expanded_data:
    for i in range(start_ind,end_ind):
        
        P[i - start_ind].append(d[i])

In [15]:
assert len(P[0]) == prev_count

In [16]:
all_symbs = set()
for i in range(99):
    all_symbs.update(P[i])
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
assert (all_symbs - set(amino_acids)) == set()

In [18]:
del P

In [19]:
# sdearch for duplicates 
from collections import defaultdict
seqs_occ = defaultdict(int)
seqs_ids = defaultdict(set)
for seq in expanded_data:
    s = ''.join(seq[start_ind:])
    seqs_occ[s] += 1
    seqs_ids[s].add(str(seq[0]))
    # print seqs_ids
    # break
assert all(seqs_occ[seq] == len(seqs_ids[seq]) for seq in seqs_ids.keys())

In [20]:
del seqs_occ, seqs_ids

In [22]:
def as_csv(l):
    return ("\t".join(l))+"\n"

In [23]:
newname = "PI_DataSet_6_19_expanded.txt"
f = open(newname,"w")
f.write(as_csv(header))
count = 0
for d in expanded_data:
    t = as_csv(d)
    f.write(as_csv(d))
    count += 1
print count

414009
